# Using MoviePy for Good (and Evil\*) 

## Helper Functions

In [ ]:
from IPython.display import HTML

def display_video(url, width=500, height=500):
    return HTML("""
    <video width="%d" height="%d" style="margin:auto auto;" autoplay loop="true" >
      <source src="%s" type="video/mp4">
    </video>
    """ % (width, height, url))

## Data Visualizations

Because this is a Python talk in 2016, I'm obligated to show a plot.

You can use MoviePy for data visualizations. Below is an example by [Zulko](http://zulko.github.io/blog/2014/11/29/data-animations-with-python-and-moviepy/). (MoviePy's author)
x
It shows the 2D slice of a 3D plot at each frame. This is the only plot you're going to see from me tonight.

\begin{align}
z & = sinc(x^2) + sin(x+y)
\end{align}

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from moviepy.video.io.bindings import mplfig_to_npimage
import moviepy.editor as mpy

# DRAW A FIGURE WITH MATPLOTLIB

duration = 2

fig_mpl, ax = plt.subplots(1,figsize=(5,3), facecolor='white')
xx = np.linspace(-2,2,200) # the x vector
zz = lambda d: np.sinc(xx**2)+np.sin(xx+d) # the (changing) z vector
ax.set_title("Elevation in y=0")
ax.set_ylim(-1.5,2.5)
line, = ax.plot(xx, zz(0), lw=3)

# ANIMATE WITH MOVIEPY (UPDATE THE CURVE FOR EACH t). MAKE A GIF.

def make_frame_mpl(t):
    line.set_ydata( zz(2*np.pi*t/duration))  # <= Update the curve
    return mplfig_to_npimage(fig_mpl) # RGB image of the figure

animation =mpy.VideoClip(make_frame_mpl, duration=duration)
animation.write_videofile("./sinc_mpl.mp4", fps=60)

display_video('sinc_mpl.mp4', 512, 384)

## Using PyGame

Sometimes you need somthing more advanced than rendering a mask or text on a video. As long as you can generate an 2D NumPy array of colors, you can anything!

Here's an example using PyGame:

In [ ]:
import os
import pygame
import moviepy.editor as mpy
from IPython.display import HTML

# Headless SDL
os.environ['SDL_VIDEODRIVER'] = 'dummy'

# Start up pygame
pygame.init()

SCREEN_SIZE = (1024, 768)
# Going to hide the main display (make it small)
pygame.display.set_mode((1,1))

# Buffer we're going to draw changes in
screen = pygame.Surface(SCREEN_SIZE, pygame.SRCALPHA, 32)

# Get your duck and pipe
duckface = pygame.image.load("res/bird.png").convert_alpha()
pipe = pygame.image.load("res/pipe.png").convert_alpha()

# Scale the pipe and move it
scale = 5
w,h = pipe.get_size()
pipe = pygame.transform.scale(pipe, (int(w*scale), int(h*scale)))
pipe_flipped = pygame.transform.flip(pipe, False, True)

# Rectangles we will use to render our sprites
duck_position = duckface.get_rect()
pipe_position = pipe.get_rect().move(0, 350)
dx = 5
dy = 3

# initialize font; must be called after 'pygame.init()' to avoid 'Font not Initialized' error
text_font = pygame.font.SysFont("Impact", 200)


# Function to get a frame 
# Returns a NumPy-style array size WIDTHxHEIGHTx3
# Represeting a 2D RGB Array. (Each component is 8-bit)

def next_frame(dt):
    global duck_position # globals, lol
    
    # Translate by (2, 0)
    duck_position = duck_position.move(dx, dy)

    # Fill the screen with skyblue
    screen.fill(0xFF87CEEB)

    angle = (-1 * float(duck_position.y)/SCREEN_SIZE[0] * 360)
    duckface_rotated = pygame.transform.rotate( duckface,angle )
    # Draw duckface
    screen.blit(duckface_rotated, duck_position)
    
    # Draw the pipes
    for i in range(0, 8):
        x = i * 120
        screen.blit(pipe, pipe_position.move(x, x/2))    

    screen.blit(pipe_flipped, pipe_position.move(800, -700))  
 
    # Draw our label
    label = text_font.render(u"θ:%.3g°" % angle, 1, (255,255,255))    
    screen.blit(label, (30, 30))
    
    # Bring the image from the other buffer to 
    # main
    pygame.display.flip()

    # For some reason, we have to do this or
    # else the image comes out flipped
    newscreen = pygame.transform.rotate(screen, 90)

    # Reverse the columns of the pixel array
    # We could do it with MoviePy with .fx(vfx.mirror_y)
    return pygame.surfarray.array3d(newscreen)[:,::-1]
    

duration = 2 # in seconds
animation = mpy.VideoClip(next_frame, duration=duration)
animation.write_videofile("./video.mp4", fps=60) 


display_video('video.mp4')


## Making Contemporary Video (GIF Compositing)


In [ ]:
HTML("""
<div style="width:600px; margin:0 auto;"><img  src="./res/datboi.gif" width="200" height="200" style="display:block;float:left;">
<video width="300" height="300" autoplay loop="true" >
  <source src="./res/sunset.mp4" type="video/mp4">
</video></div>""")

In [ ]:
# Import everything needed to edit video clips
from moviepy.editor import *


# Because it's a transparent GIF, it has a mask.
clip = VideoFileClip("res/datboi.gif", has_mask=True).subclip(0,.3)
sunset = VideoFileClip("res/sunset.mp4").subclip(0,.3)

video = CompositeVideoClip([sunset, clip])

# Write the result to a file
video.write_videofile("datboi_over_sunset.mp4",fps=24)

display_video('datboi_over_sunset.mp4')